## Setup

In [ ]:
ISRUN3 = True
isData = True # use for NuWro or NuMI data 
NUE_INTRINSIC = True

In [ ]:
# plot event rate variations, fractional uncertainties, & data/MC comparisons 
# for all sources of systematic error
# also consider potential NuMI oscillations on the event rate 
# make sure to update the plots_path here & in backend function scripts before saving

In [ ]:
import sys

sys.path.insert(0, 'backend_functions')

import selection_functions as sf

import importlib

import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb


import awkward
import matplotlib.pyplot as plt
import pandas as pd

import ROOT

import top 
from top import *

import uncertainty_functions 
from uncertainty_functions import *

import xsec_functions 
from xsec_functions import smear_matrix

from ROOT import TH1D, TH2D, TDirectory, TH1F, TH2F

from selection_functions import *


In [ ]:
from datetime import datetime
import time
now = datetime.now()
date_time = now.strftime("%H:%M:%S")
print("date and time:",date_time)

In [ ]:
import NuMIGeoWeights
importlib.reload(NuMIGeoWeights)

if ISRUN3: 
    current = "RHC"
    
else: 
    current = "FHC"

numiBeamlineGeoWeights = NuMIGeoWeights.NuMIGeoWeights(current=current) 

In [ ]:
import NuMIDetSys
importlib.reload(NuMIDetSys)

NuMIDetSysWeights = NuMIDetSys.NuMIDetSys()

In [ ]:
plots_path = parameters(ISRUN3)['plots_path']
plots_path

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

DATA = ""
EXT = ""
OVRLY  = ""
DRT = ""
NUE = ""


# slimmed with opening angle 
path = parameters(ISRUN3)['cv_ntuple_path']
print('path = ', path)

if not ISRUN3: 
    
    # Run 1 FHC 
    OVRLY = 'neutrinoselection_filt_run1_overlay_v7'
    EXT = 'neutrinoselection_filt_run1_beamoff_v5'
    DATA = 'neutrinoselection_filt_run1_beamon_beamgood_v5'
    DRT = 'prodgenie_numi_uboone_overlay_dirt_fhc_mcc9_run1_v28_all_snapshot'
    
    if NUE_INTRINSIC: 
        NUE = 'neutrinoselection_filt_run1_overlay_intrinsic_v7'

else: 
    
    # Run 3 RHC
    OVRLY = 'neutrinoselection_filt_run3b_overlay_v7'
    DATA = 'neutrinoselection_filt_run3b_beamon_beamgood_v5'
    EXT = 'neutrinoselection_filt_run3b_beamoff_v5'
    DRT = 'neutrinoselection_filt_run3b_dirt_overlay_v6'
    
    if NUE_INTRINSIC: 
        NUE = 'neutrinoselection_filt_run3b_overlay_intrinsic_v7'



In [ ]:
#overlay = uproot.open("/uboone/data/users/kmiller/uBNuMI_CCNp/ntuples/run1/cv/test/neutrinoselection_filt_run1_overlay_v64_eventweight_v2_slim.root")[fold][tree]
overlay = uproot.open(path+OVRLY+".root")[fold][tree]
data = uproot.open(path+DATA+".root")[fold][tree]
ext = uproot.open(path+EXT+".root")[fold][tree]
dirt = uproot.open(path+DRT+".root")[fold][tree]  

uproot_v = [overlay,data,ext,dirt]

if NUE_INTRINSIC: 
    nue = uproot.open(path+NUE+".root")[fold][tree]
    uproot_v.append(nue)


In [ ]:
variables = [
    "trk_score_v", 
    "shr_tkfit_dedx_Y", 
    "n_tracks_contained", 
    "NeutrinoEnergy2",
    "run","sub","evt",
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "shrsubclusters0","shrsubclusters1","shrsubclusters2",
    "trkshrhitdist2",
    "n_showers_contained", 
    "shr_phi", "trk_phi", "trk_theta",
    "shr_score", 
    "trk_energy", 
    "tksh_distance", "tksh_angle",
    "shr_energy_tot_cali", "shr_energy_cali", 
    "nslice", 
    "contained_fraction",
    "shrmoliereavg", "shr_px", "shr_py", "shr_pz", "swtrig_pre"
]

# MC only variables
mc_var = ["nu_pdg", "shr_theta", "true_e_visible", "ccnc", 
          "nproton", "nu_e", "npi0", "npion",
          "true_nu_vtx_x", "true_nu_vtx_y" , "true_nu_vtx_z", 
          "weightTune", "weightSpline", "weightSplineTimesTune", 
          "true_nu_px", "true_nu_py", "true_nu_pz", 
          "elec_e", "proton_e", "mc_px", "mc_py", "mc_pz", "elec_px", "elec_py", "elec_pz", 
          "ppfx_cv", "mc_pdg", "opening_angle"]

sys_genie = ["weightsGenie", "weightsReint", 
             "knobRPAup", "knobRPAdn", 
             "knobCCMECup", 
             "knobAxFFCCQEup", 
             "knobVecFFCCQEup", 
             "knobDecayAngMECup", 
             "knobThetaDelta2Npiup", 
             "knobThetaDelta2NRadup", 
             #"knobRPA_CCQE_Reducedup", "knobRPA_CCQE_Reduceddn", # obsolete
             "knobNormCCCOHup", 
             "knobNormNCCOHup",   
             "knobxsr_scc_Fv3up",  # these are supposed to be multisims - 10 universes each -- map to pull out
             "knobxsr_scc_Fa3up"
            ]

sys_flux = ['weightsPPFX']

## Create pandas dataframes

In [ ]:
overlay = overlay.pandas.df(variables + mc_var + sys_genie + sys_flux, flatten=False)

In [ ]:
dirt = dirt.pandas.df(variables + mc_var + sys_genie[:-2] + sys_flux, flatten=False)

In [ ]:
dirt['knobxsr_scc_Fv3up'] = 1
dirt['knobxsr_scc_Fa3up'] = 1

In [ ]:
if NUE_INTRINSIC: 
    nue = nue.pandas.df(variables + mc_var + sys_genie + sys_flux, flatten=False)

In [ ]:
data = data.pandas.df(variables, flatten=False) 

In [ ]:
ext = ext.pandas.df(variables, flatten=False)

In [ ]:
for var in mc_var+sys_genie+sys_flux: 
    data[var] = np.nan
    ext[var] = np.nan

In [ ]:
# is dirt bool

overlay['isDirt'] = False
dirt['isDirt'] = True

if NUE_INTRINSIC: 
    nue['isDirt'] = False
    
data['isDirt'] = np.nan
ext['isDirt'] = np.nan

In [ ]:
# how to get the LLR-PID value for the "track candidate" 
# (proton for nue selection, muon for numu)
# can be done for any variable
# code from Giuseppe!
#LLR-PID : log likelihood ratio particle ID 

df_v = [overlay,data,ext,dirt]

if NUE_INTRINSIC: 
    df_v.append(nue)
    
for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    
    df['NeutrinoEnergy2_GeV'] = df['NeutrinoEnergy2']/1000
    


In [ ]:
overlay = overlay.query('swtrig_pre==1')
dirt = dirt.query('swtrig_pre==1')
nue = nue.query('swtrig_pre==1')

In [ ]:
mc_df = [overlay, dirt]

if NUE_INTRINSIC: 
    mc_df.append(nue)
    

In [ ]:
# Add truth level theta & phi angles (detector & beam coordinates)
overlay = addAngles(overlay)
dirt = addAngles(dirt)

if NUE_INTRINSIC: 
    nue = addAngles(nue)

In [ ]:
for i,df in enumerate(mc_df):
    # is signal bool 
    df['is_signal'] = np.where((df.swtrig_pre == 1) 
                             & (df.nu_pdg==12) & (df.ccnc==0) & (df.nproton>0) & (df.npion==0) & (df.npi0==0)
                             & (10 <= df.true_nu_vtx_x) & (df.true_nu_vtx_x <= 246)
                             & (-106 <= df.true_nu_vtx_y) & (df.true_nu_vtx_y <= 106)
                             & (10 <= df.true_nu_vtx_z) & (df.true_nu_vtx_z <= 1026), True, False)

In [ ]:
for i,df in enumerate(mc_df):
  
    # get right order of magnitude for multiverses
    df['weightsPPFX'] = df['weightsPPFX']/1000
    df['weightsReint'] = df['weightsReint']/1000
    df['weightsGenie'] = df['weightsGenie']/1000
    
    # add beamline geometry weights
    df['weightsNuMIGeo'] = df.apply( lambda x: numiBeamlineGeoWeights.calculateGeoWeight(x['nu_pdg'],x['nu_e'],x['thbeam']) , axis=1)
 

In [ ]:
# make dataframes equal # of columns 

data['is_signal'] = np.nan
ext['is_signal'] = False

nan_var = ['thdet', 'phidet', 'true_nu_px_beam', 'true_nu_py_beam', 'true_nu_pz_beam', 
           'thbeam', 'phibeam','weightsNuMIGeo']

for var in mc_var+sys_genie+sys_flux+nan_var: 
    data[var] = np.nan
    ext[var] = np.nan

In [ ]:
# np.setdiff1d(ext.columns,overlay.columns)
#ext.columns == overlay.columns

In [ ]:
# some checks 
print(len(nue.query('is_signal==True'))==len(nue.query(signal)))
print(len(nue.query('is_signal==False'))==len(nue.query(not_signal)))

In [ ]:
ISRUN3

## Weights

In [ ]:
# clean bad weights & values 

for i,df in enumerate(mc_df):
    
    print(i)
    
    # bad weights 
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 60, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.
    
    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 60, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.  
    
                
    # CLEAN GENIE UNISIM WEIGHTS & CREATE WEIGHTSGENIEUNISIM LIST 
    for v in sys_genie[2:]: 
        df.loc[ df[v] <= 0, v ] = 1.
        df.loc[ df[v] == np.inf, v ] = 1.
        df.loc[ df[v] > 60, v ] = 1.
        df.loc[ np.isnan(df[v]) == True, v ] = 1.
        
    universes = []
    for evt in df[sys_genie[2:]].values: 
        universes.append( evt )
            
    df['weightsGenieUnisim'] = universes

        
    # cleaning -- for entries that are arrays 
    for ievt in range(df.shape[0]):
        
        # GENIE MULTISIMS
        
        # check for NaNs separately        
        if np.isnan(df['weightsGenie'].iloc[ievt]).any() == True: 
            df['weightsGenie'].iloc[ievt][ np.isnan(df['weightsGenie'].iloc[ievt]) ] = 1.
            
        reweightCondition = ((df['weightsGenie'].iloc[ievt] > 60) | (df['weightsGenie'].iloc[ievt] < 0)  | 
                             (df['weightsGenie'].iloc[ievt] == np.inf) | (df['weightsGenie'].iloc[ievt] == np.nan))
        df['weightsGenie'].iloc[ievt][ reweightCondition ] = 1.
        
        # if no variations exist for the event
        if not list(df['weightsGenie'].iloc[ievt]): 
            df['weightsGenie'].iloc[ievt] = [1.0 for k in range(600)]
        
        # RE-INTERACTION WEIGHTS
        
        # check for NaNs separately        
        if np.isnan(df['weightsReint'].iloc[ievt]).any() == True: 
            df['weightsReint'].iloc[ievt][ np.isnan(df['weightsReint'].iloc[ievt]) ] = 1.
        
        reweightCondition2 = ((df['weightsReint'].iloc[ievt] > 60) | (df['weightsReint'].iloc[ievt] < 0)   |
                             (df['weightsReint'].iloc[ievt] == np.inf))
        df['weightsReint'].iloc[ievt][ reweightCondition2 ] = 1.
        
        # if no variations exist for the event
        if not list(df['weightsReint'].iloc[ievt]): 
            df['weightsReint'].iloc[ievt] = [1.0 for k in range(1000)]
            

In [ ]:
# POT normalization

beamon_pot = parameters(ISRUN3)['beamon_pot'] 

overlay = pot_scale(overlay, 'overlay', ISRUN3)
#overlay['pot_scale'] = beamon_pot/2.33828e+21
dirt = pot_scale(dirt, 'dirt', ISRUN3)
ext = pot_scale(ext, 'ext', ISRUN3)

if NUE_INTRINSIC: 
    nue = pot_scale(nue, 'intrinsic', ISRUN3)

data['pot_scale'] = [1 for x in range(len(data))]

In [ ]:
# total weights 

# combined genie * POT weight * flux weight 
# ext gets POT weight only 

################################################################
# totweight_data scales to BEAMON

# tuned
overlay['totweight_data'] = overlay['pot_scale']*overlay['ppfx_cv']*overlay['weightSplineTimesTune']
dirt['totweight_data'] = dirt['pot_scale']*dirt['ppfx_cv']*dirt['weightSplineTimesTune']

if NUE_INTRINSIC: 
    nue['totweight_data'] = nue['pot_scale']*nue['ppfx_cv']*nue['weightSplineTimesTune']



In [ ]:
# to keep the number of columns the same 

new_var = ['weightsGenieUnisim', 'totweight_data']

for var in new_var: 
    ext[var] = np.nan
    data[var] = np.nan

## Categories

In [ ]:
# replace overlay nue CC events with nue intrinsic sample

if NUE_INTRINSIC: 
    
    # intrinsic sample contains in AV TPC events ONLY, & only CC events (overlay is entire cryo)
    print("# of nueCC in AV in overlay sample = "+str(len(overlay.query(nueCC_query))))
    len1 = len(overlay)
    
    idx = overlay.query(nueCC_query).index
    overlay.drop(idx, inplace=True)
    len2 = len(overlay) 
    print("# of nueCC in AV dropped in overlay = "+str(len1-len2))
    
    overlay = pd.concat([overlay,nue], ignore_index=True)

    # from here on out everything else should be the same. 


In [ ]:
# apply SW trigger, combine overlay + dirt as MC 
mc = pd.concat([overlay.query('swtrig_pre==1'),dirt.query('swtrig_pre==1')], ignore_index=True, sort=True)


In [ ]:
# separate by in/out FV & cosmic
infv = mc.query(in_fv_query)
outfv = mc.query(out_fv_query)

In [ ]:

# check that everything is accounted for 
print(len(mc)==len(infv)+len(outfv))#+len(cosmic))

if not (len(mc)==len(infv)+len(outfv)):#+len(cosmic)): 
    d = len(mc) - (len(infv)+len(outfv))#+len(cosmic))
    print(d)
    
     
    m = pd.concat([infv, outfv]) #pd.concat([infv, cosmic, outfv])
    diff = np.setdiff1d(list(mc.index),list(m.index))


In [ ]:
tot_signal_weighted = np.nansum(infv.query('is_signal==True')['pot_scale'])
print('total signal events = '+ str(tot_signal_weighted))


In [ ]:
print('total generated signal events = ', sum(generated_signal(ISRUN3, 'nu_e', 1, 0, 100)[0]))

In [ ]:
# 5 main categories: 

# infv - overlay & dirt events with truth vtx in FV 
# outfv - overlay & dirt events with truth vtx in FV that are classified as neutrinos
# cosmic - overlay & dirt events with true vtx in FV that get misclassified as cosmic - no longer used 
# ext - beam OFF data
# data - beam ON data 

datasets = {
    "infv": infv, 
    "outfv": outfv, 
    "ext": ext,
    "data": data
}


In [ ]:
ISRUN3

# Apply BDT Model 

In [ ]:
useBDT = True

In [ ]:
if useBDT: # load bdt model 
    bdt_model = xgb.Booster({'nthread': 4})
    bdt_model.load_model(parameters(ISRUN3)['bdt_model'])

In [ ]:
if useBDT: 
    
    datasets_bdt = {}

    for i in range(len(datasets)): 

        df = list(datasets.values())[i].copy()
        df = df.query(BDT_LOOSE_CUTS)

        # clean datasets 
        for column in training_parameters:
            df.loc[(df[column] < -1.0e37) | (df[column] > 1.0e37), column] = np.nan

        # create testing dmatrix 
        df_test = xgb.DMatrix(data=df[training_parameters])

        # apply the bdt selection
        preds = bdt_model.predict(df_test)

        # add columns for plotting 
        df['BDT_score'] = preds

        datasets_bdt[list(datasets.keys())[i]] = df

  
    

In [ ]:
if useBDT: 
    bdt_score_cut = parameters(ISRUN3)['bdt_score_cut']

    print("BDT SCORE THRESHOLD = "+str(bdt_score_cut))

    selected_query = BDT_LOOSE_CUTS + ' and BDT_score>'+str(bdt_score_cut)
    selected_signal_query = selected_query + ' and is_signal==True'

In [ ]:
# stat only errors 
if useBDT: 
    x = plot_mc('BDT_score', [x*0.1 for x in range(11)], 0, 1, BDT_LOOSE_CUTS, datasets_bdt, 
                ISRUN3, x_label="BDT Score", norm='data')

In [ ]:
ISRUN3

# Uncertainties

In [ ]:
background_subtraction = False
detsys = True

In [ ]:
plot_variations = False
plot_cov = False

datasets_dict = datasets_bdt
print("make sure to update the datasets dictionary used!")

q = BDT_LOOSE_CUTS +' and BDT_score>' + str(parameters(ISRUN3)['bdt_score_cut'])

#'nslice==1 and '+reco_in_fv_query+' and contained_fraction>0.9 and n_tracks_contained>0'

#BDT_LOOSE_CUTS + ' and shr_energy_tot_cali>0.07 and -0.9<tksh_angle<0.9'
#BDT_LOOSE_CUTS #+' and BDT_score>' + str(parameters(ISRUN3)['bdt_score_cut'])


In [ ]:
ISRUN3

In [ ]:
xvar = 'tksh_angle'
true_var = 'opening_angle'

bins = [-1, -0.5, np.cos(100 * np.pi/180), np.cos(80 * np.pi/180), 0.5, 1]
x_ticks = [-1, -0.5, -0.174, 0.174, 0.5, 1]


xlow = bins[0]
xhigh = bins[-1]

x_label = "cos $\\theta_{ep}$"

if ISRUN3: 
    y_label = '$\\nu$ / 5.0 $\\times 10^{20}$ POT'
else: 
    y_label = '$\\nu$ / 2.0 $\\times 10^{20}$ POT'


In [ ]:
# stat only -- SCALES TO DATA

full_event_rate = plot_mc(xvar, bins, xlow, xhigh, q, datasets_dict, ISRUN3, 
            norm='data', save=False, 
                         x_label=x_label, y_label=y_label)['CV']
full_event_rate

In [ ]:
bkgd_event_rate = plot_mc(xvar, bins, xlow, xhigh, q, datasets_dict, ISRUN3, 
            norm='data', x_label=x_label, save=False)['background_counts']

In [ ]:
ISRUN3

### Background subtraction

In [ ]:
if background_subtraction: # use the estimated signal event rate only 
    evt_rate = [x-y for x,y in zip(full_event_rate,bkgd_event_rate)]
    
else: 
    evt_rate = full_event_rate


In [ ]:
evt_rate == full_event_rate

### PPFX

In [ ]:
ISRUN3

In [ ]:
ncv_nu, ppfx_variations = plotSysVariations(xvar, true_var, bins, xlow, xhigh, q, datasets_dict, 'weightsPPFX',
                                         600,ISRUN3, plot=plot_variations, axis_label='Reco '+x_label, 
                                         pot=str(beamon_pot)+" POT", background_subtraction=background_subtraction)

In [ ]:
ppfx_dict = calcCov(xvar, bins, ncv_nu, evt_rate, ppfx_variations, plot=plot_cov, save=False, 
                    axis_label='Reco '+x_label, pot=str(beamon_pot)+" POT", isrun3=ISRUN3)

In [ ]:
ppfx_dict['fractional_uncertainty']

### Beamline Geometry

In [ ]:
# ordered by beamline variation run number
# [+1sigma run #, -1sigma run #]

beamline_variations = []

beamline_runs = {
    'HornCurrent' : [1, 2], 
    'xHorn1' : [3, 4], 
    'yHorn1' : [5, 6], 
    'BeamSpotSize' : [7, 8], 
    'xHorn2' : [9, 10], 
    'yHorn2' : [11, 12], 
    'WaterOnHorns' : [13, 14], 
    'xBeamShift' : [15, 16], 
    'yBeamShift' : [17, 18], 
    'zTargetPosition' : [19, 20]    
}

beamline_cov = {}

# index in weightsNuMIGeo are offset by -1

for variation in beamline_runs.keys(): 
    
    idx = [i-1 for i in beamline_runs[variation]]
    print(idx)
    
    ncv_nu, variations = plotSysVariations(xvar, true_var, bins, xlow, xhigh, q, datasets_dict, 'weightsNuMIGeo', 
                                                 idx, ISRUN3, plot=plot_variations, 
                                                 axis_label='Reco '+x_label, pot=str(beamon_pot)+" POT", 
                                                  background_subtraction=background_subtraction, title=variation)
    
    beamline_variations.append([list(a) for a in variations])
    
    # calc covariance 
    beamline_cov[variation] = calcCov(xvar, bins, ncv_nu, evt_rate, 
                                      variations, plot=plot_cov, save=False, 
                    axis_label='Reco '+x_label, pot=str(beamon_pot)+" POT", isrun3=ISRUN3)
    



In [ ]:
# compute total covariance, correlation, & uncertainty 

cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for variation in beamline_cov.keys(): 
    
    for i in range(len(bins)-1): 
        for j in range(len(bins)-1):
            
            cov[i][j] += beamline_cov[variation]['cov'][i][j]
            frac_cov[i][j] += beamline_cov[variation]['frac_cov'][i][j] 

            
for i in range(len(bins)-1): 
    for j in range(len(bins)-1):
        
        if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
            
beamline_dict = {
    'cov' : cov, 
    'frac_cov' : frac_cov,
    'cor' : cor,
    'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
} 

beamline_dict['fractional_uncertainty']

### GENIE multisims

In [ ]:
len(overlay.weightsGenie[0])

In [ ]:
ncv_nu, genie_variations = plotSysVariations(xvar, true_var, bins, xlow, xhigh, q, datasets_dict, 'weightsGenie', 500, 
                                         ISRUN3, plot=plot_variations, axis_label='Reco '+x_label, 
                                          pot=str(beamon_pot)+" POT", 
                                              background_subtraction=background_subtraction)

In [ ]:
genie_dict = calcCov(xvar, bins, ncv_nu, evt_rate, genie_variations, plot=plot_cov, save=False, 
                    axis_label='Reco '+x_label, pot=str(beamon_pot)+" POT", isrun3=ISRUN3)

In [ ]:
genie_dict['fractional_uncertainty']

### GENIE unisims 

In [ ]:

# divide the tune weight out of everything except SCC variations
# don't divide the tune weight out of SCC variations 

genie_unisim_variations = ['RPA', 
                           'CCMEC', 'AxFFCCQE', 'VecFFCCQE', 'DecayAngMEC', 'ThetaDelta2Npi', 'ThetaDelta2NRad', 
                          'NormCCCOH', 'NormNCCOH', 
                          'xsr_scc_Fv3', 'xsr_scc_Fa3']


genie_unisim_cov = {}

genie_us_variations = []

for knob in genie_unisim_variations: 
    
    if knob == 'RPA': 
        idx = [sys_genie[2:].index('knobRPAup'), sys_genie[2:].index('knobRPAdn')]
    
    else: 
        idx = [sys_genie[2:].index('knob'+knob+'up')]
    
    ncv_nu, variations = plotSysVariations(xvar, true_var, bins, xlow, xhigh, q, datasets_dict, 'weightsGenieUnisim', 
                                        idx, ISRUN3, plot=plot_variations, axis_label='Reco '+x_label, 
                                        pot=str(beamon_pot)+" POT", 
                                        background_subtraction=background_subtraction, title=knob)
    
    genie_us_variations.append([list(a) for a in variations])
    
    # calc covariance 
    genie_unisim_cov[knob] = calcCov(xvar, bins, ncv_nu, evt_rate, variations, plot=plot_cov, save=False, 
                    axis_label='Reco '+x_label, pot=str(beamon_pot)+" POT", isrun3=ISRUN3)
    

In [ ]:
# compute total covariance, correlation, & uncertainty 

cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for variation in genie_unisim_cov.keys(): 
    
    for i in range(len(bins)-1): 
        for j in range(len(bins)-1):
            
            cov[i][j] += genie_unisim_cov[variation]['cov'][i][j]
            frac_cov[i][j] += genie_unisim_cov[variation]['frac_cov'][i][j] 

            
for i in range(len(bins)-1): 
    for j in range(len(bins)-1):
        
        if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
        
            
genie_unisim_dict = {
    'cov' : cov, 
    'frac_cov' : frac_cov,
    'cor' : cor,
    'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
} 

genie_unisim_dict['fractional_uncertainty']

### GEANT4 

In [ ]:
ncv_nu, geant4_variations = plotSysVariations(xvar, true_var, bins, xlow, xhigh, q, datasets_dict, 'weightsReint', 1000, 
                                         ISRUN3, plot=plot_variations, axis_label='Reco '+x_label, pot=str(beamon_pot)+" POT", 
                                              background_subtraction=background_subtraction)



In [ ]:
geant4_dict = calcCov(xvar, bins, ncv_nu, evt_rate, geant4_variations, plot=plot_cov, save=False, 
                    axis_label='Reco '+x_label, pot=str(beamon_pot)+" POT", isrun3=ISRUN3)

In [ ]:
geant4_dict['fractional_uncertainty']

### Detector Systematics

#### Create ROOT file with BDT-selected detector variations 

In [ ]:
detsys_flat = True
recreate_file = False

#detvar_file = "detsys_08122022.root" 

In [ ]:
if not detsys_flat: 

    if ISRUN3: 
        detvar = detvar_run3_rhc.keys()

    else: 
        detvar = detvar_run1_fhc.keys()
        
    # skip this step if it is already created
    # should manually delete the file first 
    # (located here: /uboone/data/users/kmiller/uBNuMI_CCNp/ntuples/runX/systematics/detvar/)

    # scales to the det sys CV POT (standard overlay)

    if recreate_file: 
        for v in list(detvar): 
            NuMIDetSysWeights.makehist_detsys(v, ISRUN3, detvar_file, xvar, bins, cut=q, useBDT=True, 
                                             background_subtraction=background_subtraction)


In [ ]:
if not detsys_flat:

    detector_variations = NuMIDetSysWeights.plot_variations(xvar, bins, 
                                                            detvar_file,  
                                                            ISRUN3, axis_label=x_label, 
                                                            plot=True, background_subtraction=background_subtraction)
    

In [ ]:
# compute total covariance, correlation, & uncertainty 

cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

detsys_dict = {}

In [ ]:
# compute covariance (N=1 for each variation)
# detvar are scaled to beam on POT at this point

if not detsys_flat:

    detsys_cov = {}

    for variation in detector_variations.keys(): 

        if variation=='CV': 
            continue

        # calc covariance for each unisim 
        detsys_cov[variation] = calcCov(xvar, bins, detector_variations['CV'], detector_variations['CV'], 
                                        [detector_variations[variation]], 
                                        plot=False, save=False, axis_label='Reco '+x_label, pot=str(beamon_pot)+" POT", 
                                        isrun3=ISRUN3)
        
    for i in range(len(bins)-1): 
        for j in range(len(bins)-1):
            
            cov[i][j] = sum([detsys_cov[x]['cov'][i][j] for x in detsys_cov.keys()])
            frac_cov[i][j] = sum([detsys_cov[x]['frac_cov'][i][j] for x in detsys_cov.keys()])

    for i in range(len(bins)-1): 
        for j in range(len(bins)-1):

            if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                    cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))

    detsys_dict['cov'] = cov
    detsys_dict['frac_cov'] = frac_cov
    detsys_dict['cor'] = cor
    detsys_dict['fractional_uncertainty'] = np.sqrt(np.diag(frac_cov))


In [ ]:
if detsys_flat:
    
    detsys_dict['fractional_uncertainty'] = []
    
    for i in range(len(bins)-1): 
        
        if evt_rate[i] != 0: 
            detsys_dict['fractional_uncertainty'].append(parameters(ISRUN3)['detsys_flat'])
            frac_cov[i][i] = parameters(ISRUN3)['detsys_flat']**2
        else: 
            detsys_dict['fractional_uncertainty'].append(0)
            
                
    
    print((detsys_dict['fractional_uncertainty']))
    
    
            
            

In [ ]:
detsys_dict['frac_cov'] = frac_cov

### Stat Uncertainty of the MC event count 

In [ ]:
# if data/MC comparisons :  uncertainty on the full estimated event rate 
# if closure test : uncertainty on the MC background only  (?) 

In [ ]:
# sum of the weights squared

mc_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
mc_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

# also save sumw2 of the estimated signal and neutrino background
mc_signal_sumw2 = []
mc_bkgd_sumw2 = []


In [ ]:
# for data/mc comparisons
if background_subtraction==False: 
    
    # stat uncertainty for MC - signal + background 
    ncv = pd.concat([datasets_dict['infv'].copy().query(q), 
                             datasets_dict['outfv'].copy().query(q)], ignore_index=True)  
    
    
    

In [ ]:
if background_subtraction==True: 

    # NuWro fake data test: take sum of weights squared for MC bkgd+EXT only (add NuWro event count contribution later...)
    # beam on data: take sum of weights squared for MC bkgd+EXT only (add beam-on event count contribution later...)
    if isData==True: 
        ncv = pd.concat([datasets_dict['infv'].copy().query(q+' and is_signal==False'), 
                                         datasets_dict['outfv'].copy().query(q+' and is_signal==False')], 
                                ignore_index=True) 

    # GENIE closure test: take sum of weights squared for the full event count 
    else: 
        ncv = pd.concat([datasets_dict['infv'].copy().query(q), 
                             datasets_dict['outfv'].copy().query(q)], ignore_index=True) 



In [ ]:
for i in range(len(bins)-1):

    if i==len(bins)-2: 
        bin_query = xvar+'>='+str(bins[i])+' and '+xvar+'<='+str(bins[i+1])
    else: 
        bin_query = xvar+'>='+str(bins[i])+' and '+xvar+'<'+str(bins[i+1])
        
    mc_stat_cov[i][i] = sum(ncv.query(bin_query).totweight_data ** 2) 
    
    mc_signal_sumw2.append(sum(datasets_dict['infv'].copy().query(q+' and is_signal==True and '+bin_query).totweight_data ** 2) )
    mc_bkgd_sumw2.append( sum(pd.concat([datasets_dict['infv'].copy().query(q), 
                                        datasets_dict['outfv'].copy().query(q)], ignore_index=True).query("is_signal==False and "+bin_query).totweight_data **2 ))

    if mc_stat_cov[i][i] != 0: 
        mc_frac_stat_cov[i][i] = mc_stat_cov[i][i]/ evt_rate[i]**2 
    
    bin_query = ''
    
mc_stat_percent_error = np.sqrt(np.diag(mc_frac_stat_cov))
mc_stat_percent_error

In [ ]:
#mc_bkgd_sumw2_reweight = [a*b for a,b in zip(mc_bkgd_sumw2,nuwro_to_genie)]
#mc_bkgd_sumw2_reweight

In [ ]:
if plot_cov: 
    fig = plt.figure(figsize=(10, 6))
        
    plt.pcolor(bins, bins, mc_stat_cov, cmap='OrRd', edgecolors='k' )

    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=14)

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    plt.xlabel("Reco "+x_label, fontsize=15)
    plt.ylabel("Reco "+x_label, fontsize=15)

    plt.title('MC Statistical Covariance', fontsize=15)

    plt.show()
    
    fig = plt.figure(figsize=(10, 6))
        
    plt.pcolor(bins, bins, mc_frac_stat_cov, cmap='OrRd', edgecolors='k' )

    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=14)

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    plt.xlabel("Reco "+x_label, fontsize=15)
    plt.ylabel("Reco "+x_label, fontsize=15)

    plt.title('MC Fractional Statistical Covariance', fontsize=15)

    plt.show()


### Stat Uncertainty Beam On & EXT

In [ ]:
# EXT uses sum of the weights squared 
selected_ext = datasets_dict['ext'].copy().query(q)

In [ ]:
ext_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
ext_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    
for i in range(len(bins)-1):

    if i==len(bins)-2: 
        bin_query = xvar+' >= '+str(bins[i])+' and '+xvar+' <= '+str(bins[i+1])
    else: 
        bin_query = xvar+' >= '+str(bins[i])+' and '+xvar+' < '+str(bins[i+1])
        
    ext_stat_cov[i][i] = sum(selected_ext.query(bin_query).pot_scale ** 2) 
    
    if ext_stat_cov[i][i] != 0: 
        ext_frac_stat_cov[i][i] = ext_stat_cov[i][i]/ evt_rate[i]**2 
    
    bin_query = ''
    
ext_stat_percent_error = np.sqrt(np.diag(ext_frac_stat_cov))
ext_stat_percent_error

In [ ]:
ext_sumw2 = np.diagonal(ext_stat_cov)

In [ ]:
cv_ext = plt.hist(selected_ext[xvar], bins, range=[bins[0], bins[-1]],
         weights=selected_ext.pot_scale, color='gainsboro')[0]

plt.show()

In [ ]:
beamon_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    
selected_data_counts = plt.hist(datasets_dict['data'].query(q)[xvar], bins, range=[bins[0], bins[-1]])[0]
plt.xlim(xlow,xhigh)
plt.show()
    
if background_subtraction==True: 
    selected_data_counts = [a-b for a,b in zip(selected_data_counts,bkgd_event_rate)]
    
print(sum(selected_data_counts))
    
for i in range(len(bins)-1): 
    
    if selected_data_counts[i] != 0: 
        beamon_frac_stat_cov[i][i] = selected_data_counts[i]/(selected_data_counts[i]**2)

beamon_stat_percent_error = np.sqrt(np.diag(beamon_frac_stat_cov))
    
    
   

In [ ]:
plt.hist(datasets_dict['infv'].query(q)[xvar], bins, 
         range=[bins[0], bins[-1]], weights=datasets_dict['infv'].query(q).totweight_data)[0]
plt.xlim(xlow,xhigh)

### MC Response Matrix 

In [ ]:
# this is for the signal channel ONLY 

if background_subtraction: 

    selected_signal_df = datasets_dict['infv'].query(selected_signal_query).copy()
    selected_signal_df['seed'] = selected_signal_df.apply( lambda x: ConcatRunSubRunEvent(x['run'], x['sub'], x['evt']), axis=1 )
    selected_signal_df['weightsPoisson'] = selected_signal_df.apply( lambda x: PoissonRandomNumber(x['seed'], mean=1.0, size=1000), axis=1 )


In [ ]:
if background_subtraction: 

    fig = plt.figure(figsize=(8, 5))

    # histogram bin counts for all universes
    rmatrix_uni_counts = []

    for u in range(1000): 

        # multiply in with sys weight of universe u 
        sys_weight = list(selected_signal_df['weightsPoisson'].str.get(u))
        total_weight = [ x*y for x, y in zip(sys_weight, selected_signal_df['totweight_data']) ]

        n, b, p = plt.hist(selected_signal_df[xvar], bins, histtype='step', weights=total_weight, 
                                linewidth=0.5, color='cornflowerblue')  

        rmatrix_uni_counts.append(list(n))

    ncv, bcv, pcv = plt.hist(selected_signal_df[xvar], bins, histtype='step', 
                             weights=selected_signal_df['totweight_data'], linewidth=2, color='black')      

    plt.xticks(x_ticks, fontsize=14)
    plt.yticks(fontsize=14)
    
    plt.xlim(xlow, xhigh)

    plt.xlabel('Reco '+x_label, fontsize=15)
    plt.ylabel("", fontsize=15)

    plt.title('weightsPoisson', fontsize=16)    


    plt.show()

In [ ]:
if background_subtraction: 
    response_matrix_uncertainty = calcCov(xvar, bins, ncv, evt_rate, rmatrix_uni_counts, isrun3=ISRUN3, plot=False, 
                   save=False, axis_label='Reco '+x_label, pot=parameters(ISRUN3)['beamon_pot'])
    
    response_matrix_uncertainty['fractional_uncertainty']


### POT Counting (2%)

In [ ]:
parameters(ISRUN3)['beamon_pot'] 

In [ ]:
pot_counting = pot_unisims(xvar, evt_rate, bins, 0.02, ISRUN3, plot=plot_variations, x_label=None)

### Dirt (100%)  

In [ ]:
# selected dirt uncertainty 
# vary the dirt interactions by 100% (1 unisim) on the event rate 

selected_dirt = plt.hist(datasets_dict['outfv'].copy().query(q+' and isDirt==1')[xvar], 
                         bins, 
                        weights=datasets_dict['outfv'].copy().query(q+' and isDirt==1')['pot_scale'], 
                         color='orchid')[0]

In [ ]:
dirt_uncertainty = dirt_unisim(xvar, bins, evt_rate, selected_dirt, 1.0, isrun3=ISRUN3, plot=True, 
                               x_label=None, title=None)

In [ ]:
dirt_uncertainty['fractional_uncertainty']

In [ ]:
selected_dirt

## All Sources of Uncertainty

In [ ]:
ISRUN3

In [ ]:
frac_cov_dict = {
    'ppfx' : ppfx_dict['frac_cov'], 
    'beamline' : beamline_dict['frac_cov'], 
    'genie_ms' : genie_dict['frac_cov'], 
    'genie_us': genie_unisim_dict['frac_cov'], 
    'geant4' : geant4_dict['frac_cov'],
    'pot_counting' : pot_counting['frac_cov'], 
    'dirt' : dirt_uncertainty['frac_cov'],
    'mc_stat' : mc_frac_stat_cov, # either the full distribution or just background events 
    'ext_stat' : ext_frac_stat_cov
}


if background_subtraction: 
    frac_cov_dict['response_matrix'] = response_matrix_uncertainty['frac_cov']
    frac_cov_dict['beamon_stat'] = beamon_frac_stat_cov
    
if detsys: 
    frac_cov_dict['detector'] = detsys_dict['frac_cov']

In [ ]:
cv = evt_rate

tot_frac_cov, tot_abs_cov = plotFullCov(frac_cov_dict, xvar, cv, bins, xlow, xhigh, save=False, 
                      axis_label='Reco '+x_label, isrun3=ISRUN3, pot=str(beamon_pot)+' POT')

In [ ]:
# add ppfx & beamline geometry in quadrature
frac_cov_dict['flux'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(frac_cov_dict['ppfx'], frac_cov_dict['beamline'])]

In [ ]:
# add genie in quadrature
frac_cov_dict['genie_all'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(frac_cov_dict['genie_ms'], frac_cov_dict['genie_us'])]

In [ ]:
# add stat stuff in quadrature 

if background_subtraction: 
    
    if isData: # NuMI/NuWro studies
        frac_cov_dict['stat_all'] = [ [w+x+y+z for w,x,y,z in zip(a,b,c,d)] for a,b,c,d in zip( frac_cov_dict['beamon_stat'], frac_cov_dict['response_matrix'], frac_cov_dict['mc_stat'], frac_cov_dict['ext_stat'])]
    
    else:  # GENIE closure studies
        frac_cov_dict['stat_all'] = [ [x+y+z for x,y,z in zip(b,c,d)] for b,c,d in zip( frac_cov_dict['response_matrix'], frac_cov_dict['mc_stat'], frac_cov_dict['ext_stat'])]

# for mc/data comparisons
else: 
    frac_cov_dict['stat_all'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(frac_cov_dict['mc_stat'], frac_cov_dict['ext_stat'])]


In [ ]:
# clean away nans
v = np.array(tot_frac_cov)
v[np.isnan(v)] = 0
tot_frac_cov = v

In [ ]:
# clean away nans
v = np.array(tot_abs_cov)
v[np.isnan(v)] = 0
tot_abs_cov = v

In [ ]:
frac_unc_dict = {
    'flux' : np.sqrt(np.diagonal(frac_cov_dict['flux'])), 
    'genie' : np.sqrt(np.diagonal(frac_cov_dict['genie_all'])), 
    'geant4' : np.sqrt(np.diagonal(frac_cov_dict['geant4'])),
    'pot_counting' : np.sqrt(np.diagonal(frac_cov_dict['pot_counting'])), 
    'dirt' : np.sqrt(np.diagonal(frac_cov_dict['dirt'])),
    'stat' : np.sqrt(np.diagonal(frac_cov_dict['stat_all'])), # does not include beam on STAT 
    'total' : np.sqrt(np.diagonal(tot_frac_cov))
}

if detsys: 
    frac_unc_dict['detector'] = np.sqrt(np.diagonal(frac_cov_dict['detector']))

In [ ]:
frac_unc_dict['total']

In [ ]:
tot_unc = [0 for i in range(len(bins)-1)]

for source in frac_unc_dict.keys(): 
    
    if source=='total': 
        continue
    

    # square the list 
    squared = [x**2 for x in frac_unc_dict[source]]
    
    # add in quadrature 
    tot_unc = [a+b for a,b in zip(tot_unc, squared)]
   
tot_unc = np.sqrt(np.array(tot_unc))
tot_unc
 

In [ ]:
bincenters = 0.5*(np.array(bins)[1:]+np.array(bins)[:-1])

fig = plt.figure(figsize=(8, 5))  

# TOTAL 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Total",
        weights=frac_unc_dict['total'], linewidth=1.5, color='black')

# FLUX
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Flux", 
         weights=frac_unc_dict['flux'], color='royalblue')

# CROSS SECTION MODELS 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GENIE", 
         weights=frac_unc_dict['genie'], color='goldenrod')

#plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GENIE (us)", 
#         weights=np.sqrt(np.diag(frac_cov_dict['genie_us'])), color='goldenrod')

#plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GENIE (ms)", 
#         weights=np.sqrt(np.diag(frac_cov_dict['genie_ms'])), color='goldenrod', linestyle='--')

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GEANT4", 
         weights=frac_unc_dict['geant4'], color='green')

# DETECTOR 
if detsys: 
    plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Detector", 
         weights=frac_unc_dict['detector'], color='crimson')

# POT COUNTING 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="POT counting",
        weights=frac_unc_dict['pot_counting'], color='purple')

# DIRT 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Dirt",
        weights=frac_unc_dict['dirt'], color='brown')

# STATISTICAL 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Stat",
        weights=frac_unc_dict['stat'], color='hotpink')

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
#plt.ylim(0, .4)

plt.legend(fontsize=13, frameon=False, ncol=3)


if background_subtraction: 
    if ISRUN3: 
        plt.title("Uncertainty on the RHC Background-Selected Event Rate", fontsize=16)
    else: 
        plt.title("Uncertainty on the FHC Background-Selected Event Rate", fontsize=16)
        
else: 
    if ISRUN3: 
        plt.title("Uncertainty on the RHC Selected Event Rate (MC+EXT)", fontsize=16)
    else: 
        plt.title("Uncertainty on the FHC Selected Event Rate (MC+EXT)", fontsize=16)   

plt.show()

## Save covariance to unfolding file 

In [ ]:
if background_subtraction: 
    print("make sure to change file name!")
    
    variations_dict = {
        'evt_rate' : evt_rate, 
        'beamon_counts' : list(selected_data_counts), 
        'ppfx' : ppfx_variations, 
        'beamline' : beamline_variations, 
        'genie_ms' : genie_variations, 
        'genie_us' : genie_us_variations, 
        'geant4' : geant4_variations,
        'pot_counting' : pot_counting['variations'], 
        'dirt' : dirt_uncertainty['variations'], 
        'response_matrix' : rmatrix_uni_counts, 
        'cv_dirt' : list(selected_dirt), 
        'cv_bkgd' : bkgd_event_rate, # total background event rate (MC+EXT)
        'cv_ext' : list(cv_ext),
        'mc_bkgd_sumw2' : list(mc_bkgd_sumw2), # need to take the square root for fractional uncertainty 
        'ext_sumw2' : list(ext_sumw2), # need to take the square root for fractional uncertainty 
        'mc_signal_sumw2' : mc_signal_sumw2
    }
    

else: 
    print("make sure to change file name!")
    
    variations_dict = {
        'full_evt_rate' : evt_rate, 
        'beamon_full_evt_rate' : list(selected_data_counts), 
        'ppfx' : [list(a) for a in ppfx_variations], 
        'beamline' : beamline_variations, 
        'genie_ms' : [list(a) for a in genie_variations], 
        'genie_us' : genie_us_variations, 
        'geant4' : [list(a) for a in geant4_variations],
        'pot_counting' : pot_counting['variations'], 
        'dirt' : dirt_uncertainty['variations'], 
        'cv_dirt' : list(selected_dirt), 
        'cv_bkgd' : bkgd_event_rate, # total background event rate (MC+EXT)
        'cv_ext' : list(cv_ext), # just the EXT background 
        'mc_bkgd_sumw2' : list(mc_bkgd_sumw2), # need to take the square root for fractional uncertainty 
        'ext_sumw2' : list(ext_sumw2), # need to take the square root for fractional uncertainty 
        'mc_signal_sumw2' : mc_signal_sumw2
    }

In [ ]:
variations_dict.keys()

In [ ]:
from datetime import date  
import json
import os
    
path = 'unfolding/variations/full_event_rates/'
if ISRUN3: 
    filename = 'RHCVariations_FullEvtRate_'+xvar+"_"+date.today().strftime("%m%d%y")+".json"
    
else: 
    filename = 'FHCVariations_FullEvtRate_'+xvar+"_"+date.today().strftime("%m%d%y")+".json"
    
if os.path.exists(path+filename): 
    print(filename, " exists and is readable, need to update file name to save ! ")
        
else: 
    with open(path+filename, 'w') as f:
        json.dump(variations_dict, f)


In [ ]:
ISRUN3

## modify existing json file 

In [ ]:
import json

In [ ]:
ISRUN3

In [ ]:
if ISRUN3: 
    with open('unfolding/variations/RHCVariations_'+xvar+'.json') as f:
        variations_dict = json.load(f)
        
else: 
    with open('unfolding/variations/FHCVariations_'+xvar+'.json') as f:
        variations_dict = json.load(f)  
    

In [ ]:
variations_dict.keys()

In [ ]:
#variations_dict['beamon_counts'] = list(selected_data_counts)
#variations_dict['beamon_counts']

In [ ]:
#variations_dict['beamon_counts'] = list(selected_data_counts)
#variations_dict['beamon_counts']

In [ ]:
variations_dict.keys()

In [ ]:
if ISRUN3: 
    with open('unfolding/variations/RHCVariations_'+xvar+'.json', 'w') as f:
        json.dump(variations_dict, f)
        
else: 
    with open('unfolding/variations/FHCVariations_'+xvar+'.json', 'w') as f:
        json.dump(variations_dict, f)

In [ ]:
f.close()

## Data/MC Comparisons -- before background subtraction

In [ ]:
importlib.reload(sf)
from selection_functions import *

In [ ]:
importlib.reload(top)
from top import *

In [ ]:
x = plot_mc(xvar, bins, xlow, xhigh, q, datasets_dict, ISRUN3, 
            norm='data',
            x_label='Reconstructed '+x_label,
            save=False, 
            y_label=y_label,sys=frac_unc_dict['total'], xtext=0.9, ytext=40)

In [ ]:
q

In [ ]:
if ISRUN3: 
    chi2_label = "RHC RUN 3"
    save_label = "rhc_bdtcut_data"
    beamon_pot_str = "5.0 $\\times 10^{20}$"
    
else: 
    chi2_label = "FHC RUN 1"
    save_label = "fhc_bdtcut_data"
    beamon_pot_str = "2.0 $\\times 10^{20}$"
    
print("make sure to update save label!")
print("save label = ", save_label)

In [ ]:
## compute the chi2 

if not background_subtraction: 
    selected_data = plt.hist(datasets_dict['data'].copy().query(q)[xvar], bins)[0]
    plt.close()

    # inverse cov -- make sure to include the beam on stat covariance! 
    tot_cov = np.array(tot_frac_cov)+np.array(beamon_frac_stat_cov)

    for i in range(len(bins)-1): 
         for j in range(len(bins)-1): 
                tot_cov[i][j] = tot_cov[i][j] * evt_rate[i] * evt_rate[j]

    tot_inverse_cov = np.linalg.inv(tot_cov)

    ## check 
    plt.pcolor(bins, bins, np.matmul(tot_cov, tot_inverse_cov), cmap='OrRd', edgecolors='k')
    plt.xlim(xlow,xhigh)
    plt.ylim(xlow,xhigh)
    cbar = plt.colorbar()
    plt.show()


    chi2 = 0

    for i in range(len(bins)-1):  
        for j in range(len(bins)-1):  
                chi2 = chi2  + ( (evt_rate[i]-selected_data[i])*tot_inverse_cov[i][j]*(evt_rate[j]-selected_data[j]) )
    chi2

In [ ]:
# sys now takes an array of the total uncertainty

d = plot_data(xvar, bins, xlow, xhigh, q, datasets_dict, ISRUN3, 
                  save=False, 
                  save_label=save_label,  
                  x_label=x_label, ncol=3, ymax=50,
                  y_label=beamon_pot_str, x_ticks=x_ticks,
                  sys=frac_unc_dict['total'], 
                  text=chi2_label+"\n$\\chi^{2}$/n = "+str(round(chi2, 1))+"/"+str(len(bins)-1), 
                  xtext=.85, ytext=23.5)

## NuMI Oscillations (3+1 Model)

In [ ]:
# outdated 

x = plot_mc(xvar, [round(0.01*x, 2) for x in range(0, 75, 5)], 0, 0.7, 'BDT_score>0.575', datasets_bdt, ISRUN3, 
        plt_norm='proj', pot='$9.23\\times10^{20}$', ymax=30, x_label='True Neutrino Energy [GeV]', 
            osc='machado_bestfit.csv')

# osc='biggest_variation.csv'

#### Create projected oscillation dictionary 

In [ ]:
import json

In [ ]:
# to load a stored dictionary 
with open('outdated/FHC_Projected_TrueNeutrinoEnergy.json') as f:
    d = json.load(f)

In [ ]:
bins = np.linspace(0, 4.5, 46) #d['bins']

In [ ]:
x = plot_mc('nu_e', bins, 0, 5, selected_query, datasets_bdt, 
            ISRUN3, x_label="Reco $\\nu$ Energy [GeV]", norm='data', pot='$2.0\\times10^{20}$')

In [ ]:
oscillation_dict = {}

In [ ]:
oscillation_dict['bins'] = bins

In [ ]:
pot_scale = 9.23E20/parameters(ISRUN3)['beamon_pot']
print(pot_scale)

In [ ]:
oscillation_dict['CV'] = [k*pot_scale for k in x['CV']]

In [ ]:
ncv, ppfx_variations = plotSysVariations('nu_e', 'nu_e', bins, bins[0], bins[-1], selected_query, datasets_bdt, 'weightsPPFX',600, 
                                         ISRUN3, plot=False, axis_label='True Neutrino Energy [GeV]', pot='$2.0 x 10^{20}$ POT', 
                                              background_subtraction=False)

ppfx_dict = calcCov('nu_e', bins, ncv, ppfx_variations, plot=False, save=False, 
                    axis_label='True Neutrino Energy [GeV] ', pot='$2.0 x 10^{20}$ POT', isrun3=ISRUN3, title='Hadron Production')

In [ ]:
oscillation_dict['ppfx_cov_frac'] = ppfx_dict['frac_cov']

In [ ]:
ncv, genie_variations = plotSysVariations('nu_e', 'nu_e', bins, bins[0], bins[-1], selected_query, datasets_bdt, 'weightsGenie',600, 
                                         ISRUN3, plot=True, axis_label='True Neutrino Energy [GeV]', pot='$2.0 x 10^{20}$ POT', 
                                              background_subtraction=False)

genie_dict = calcCov('nu_e', bins, ncv, genie_variations, plot=False, save=False, 
                    axis_label='True Neutrino Energy [GeV] ', pot='$2.0 x 10^{20}$ POT', isrun3=ISRUN3, title='Hadron Production')

In [ ]:
oscillation_dict['genie_cov_frac'] = genie_dict['frac_cov']

In [ ]:
ncv, geant4_variations = plotSysVariations('nu_e', 'nu_e', bins, bins[0], bins[-1], selected_query, datasets_bdt, 'weightsReint',1000, 
                                         ISRUN3, plot=True, axis_label='True Neutrino Energy [GeV]', pot='$2.0 x 10^{20}$ POT', 
                                              background_subtraction=False)


geant4_dict = calcCov('nu_e', bins, ncv, geant4_variations, plot=False, save=False, 
                    axis_label='True Neutrino Energy [GeV] ', pot='$2.0 x 10^{20}$ POT', isrun3=ISRUN3, title='Hadron Production')

In [ ]:
oscillation_dict['reint_cov_frac'] = geant4_dict['frac_cov']

In [ ]:
## detector variations -- make new file 
recreate_file=True

In [ ]:
if recreate_file: 
    for v in list(detvar_run1_fhc.keys()): 
        NuMIDetSysWeights.makehist_detsys(v, ISRUN3, "NuMI_FHC_BDT_DetectorVariations_OscillationAnalysis_v2.root", 'nu_e', 
                                          bins, cut=selected_query, useBDT=True)

In [ ]:
detector_variations = NuMIDetSysWeights.plot_variations('nu_e', bins, "NuMI_FHC_BDT_DetectorVariations_OscillationAnalysis_v2.root", 
                                                        ISRUN3, axis_label='True Neutrino Energy', plot=True, background_subtraction=False)

In [ ]:
# compute covariance (N=1 for each variation)

detsys_cov = {}

# index in weightsNuMIGeo are offset by -1

for variation in detector_variations.keys(): 
    
    if variation=='CV': 
        continue
    
    # calc covariance for each unisim 
    detsys_cov[variation] = calcCov('nu_e', bins, detector_variations['CV'], [detector_variations[variation]], 'Detector', 
                                    plot=False, save=False, pot='$2.0 x 10^{20}$ POT', isrun3=ISRUN3,
                                   title=variation)

In [ ]:
# compute total covariance, correlation, & uncertainty 

cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for variation in detsys_cov.keys(): 
    
    for i in range(len(bins)-1): 
        for j in range(len(bins)-1):
            
            cov[i][j] = sum([detsys_cov[x]['cov'][i][j] for x in detsys_cov.keys()])
            
            if detector_variations['CV'][i]*detector_variations['CV'][j] != 0: 
                frac_cov[i][j] = cov[i][j]/(detector_variations['CV'][i]*detector_variations['CV'][j])

            
for i in range(len(bins)-1): 
    for j in range(len(bins)-1):
        
        if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
            
detsys_dict = {
    'cov' : cov, 
    'frac_cov' : frac_cov,
    'cor' : cor,
    'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
} 

In [ ]:
oscillation_dict['det_cov_frac'] = detsys_dict['frac_cov']

In [ ]:
tot_frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for source in list(oscillation_dict.keys())[2:]: 
    tot_frac_cov = [ [x+y for x,y in zip(a,b)] for a,b in zip(tot_frac_cov, oscillation_dict[source])]

In [ ]:
oscillation_dict['tot_cov_frac'] = tot_frac_cov

In [ ]:
oscillation_dict.keys()


In [ ]:
oscillation_dict['bins'] = oscillation_dict['bins'].tolist()

In [ ]:
# save this dictionary 

with open('mun/FHC_Projected_TrueNeutrinoEnergy_March2022_v2.json', 'w') as f:
    json.dump(oscillation_dict, f)

In [ ]:
evt_rate

In [ ]:
variations_dict['beamon_counts']

In [ ]:
variations_dict.keys()

## background subtracted event rate comparison

In [ ]:
bincenters = 0.5*(np.array(x_ticks+[xhigh])[1:]+np.array(x_ticks+[xhigh])[:-1])

In [ ]:
x_err = []
for x in range(len(bincenters)):
    print(x)
    x_err.append(round(abs((x_ticks+[3][:-1]+[3])[x+1]-(x_ticks+[3][:-1]+[3])[x])/2, 2))

In [ ]:
bincenters

In [ ]:
fig = plt.figure(figsize=(8, 5))







In [ ]:
fig = plt.figure(figsize=(8, 7))

gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])
    
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
    
ax1.tick_params(axis = 'both', which = 'major', labelsize = 13)
ax2.tick_params(axis = 'both', which = 'major', labelsize = 13)
    
ax2.yaxis.grid(linestyle="--", color='black', alpha=0.2)
ax2.xaxis.grid(linestyle="--", color='black', alpha=0.2)

ax1.set_xticks(bins[:-1])
ax2.set_xticks(bins[:-1])

ax1.set_xlim(bins[0], 3)
ax2.set_xlim(bins[0], 3)

n, b, p = ax1.hist(bincenters, bins, histtype='bar', weights=variations_dict['evt_rate'], color='orange', alpha=0.5,
            label='GENIE: '+str(round(sum(evt_rate), 1)))

ax1.errorbar(bincenters, variations_dict['beamon_counts'], yerr=np.sqrt(np.diag(tot_abs_cov)), xerr=x_err, 
             color="black", fmt='o', markersize=3, label='DATA: '+str(int(sum(variations_dict['beamon_counts']))))


ax1.step(list(bins)+[0], [0, n[0], n[-1], 0], 
         color='saddlebrown', linewidth=1, alpha=0.85)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)


ax2.set_xlabel("Reconstructed Visible Energy [GeV]", fontsize=15)

plt.xlim(bins[0], 3)

ax2.errorbar(bincenters, variations_dict['beamon_counts']/n, 
             yerr=get_ratio_err(variations_dict['beamon_counts'], n), xerr=x_err, color="black", fmt='o')
ax2.set_ylim(0, 2)
ax2.set_ylabel("DATA / GENIE", fontsize=15)
ax2.axhline(1.0, color='black', lw=1, linestyle='--')


if ISRUN3==False: 
    ax1.set_title("FHC Background-Subtracted Event Rate", fontsize=15)
    ax1.set_ylabel("$\\nu$ / 2.0 $\\times 10^{20}$ POT", fontsize=15)
else: 
    ax1.set_title("RHC Background-Subtracted Event Rate", fontsize=15)
    ax1.set_ylabel("$\\nu$ / 5.0 $\\times 10^{20}$ POT", fontsize=15)
    
ax1.legend(frameon=False, fontsize=14)


plt.show()

In [ ]:
sum(variations_dict['beamon_counts'])/sum(n)